<h1>Import necessary libraries

In [13]:
import pandas as pd
import psycopg2 as pg
import pandas.io.sql as psql
import matplotlib as plt

%matplotlib inline

<h1>Connect to DB

In [14]:
conn = pg.connect(" dbname='dev' host='njoy-testing-env.c2ag7j9q24ns.us-east-1.rds.amazonaws.com' user='reader' port='5416' password='MbmeL8ku6x'")
df = psql.read_sql('SELECT count(1) from public.orders', conn)
df.head()

,count
0,201570


In [15]:
df = psql.read_sql('SELECT * from public.orders', conn)

<h1>Preview of DataFrame

In [44]:
df.head(5)

,order_id,customer_id,created_at,updated_at,closed_at,processed_at,status,number,total_price,total_less_tax,total_tax,payment_status,closed_hour
0,8.974965e+11,230824c2,2019-03-13 18:14:55,2019-03-13 22:01:40,2019-03-13 22:01:39,2019-03-13 18:14:55.000000,fulfilled,1029702,18.28,17.0,1.28,paid,22.0
1,8.975937e+11,7100545b,2019-03-13 19:33:50,2019-03-13 23:01:21,2019-03-13 23:01:21,2019-03-13 19:33:50.000000,fulfilled,1029768,9.00,9.0,0.00,paid,23.0
2,8.975785e+11,3de1dc29,2019-03-13 19:20:05,2019-03-13 23:01:33,2019-03-13 23:01:33,2019-03-13 19:20:05.000000,fulfilled,1029757,18.02,17.0,1.02,paid,23.0
3,8.976117e+11,cb5e4189,2019-03-13 19:50:09,2019-03-13 23:01:12,2019-03-13 23:01:11,2019-03-13 19:50:09.000000,fulfilled,1029780,9.63,9.0,0.63,paid,23.0
4,8.976406e+11,579c3141,2019-03-13 20:17:01,2019-03-14 00:01:23,2019-03-14 00:01:23,2019-03-13 20:17:01.000000,fulfilled,1029803,9.26,9.0,0.26,paid,0.0


In [73]:
df.shape

(201570, 13)

<h1>Query 1</h1>
<h6>We need basic insight into hourly sales performance. We'd like a query that shows the following metrics by hour: <br>
revenue (total_price less total_tax)<br>
total number of orders<br>
number of orders placed by a customer who has previously placed an order<br>

In [156]:
#create column to seperate closing hour of orders
df['closed_hour'] = df['closed_at'].dt.hour

In [201]:
hourly_orders = df.groupby('closed_hour')['total_less_tax'].agg(['sum','count']).reset_index()

In [86]:
customer_counts = df.groupby('customer_id').count().reset_index().sort_values('order_id', ascending = False)

In [93]:
#there are 181,105 total customers in the dataframe
customer_counts.shape

(181105, 13)

In [165]:
#there are 19,339 returning customers
returning_customers = customer_counts.loc[customer_counts['order_id'] > 1][['customer_id', 'order_id']]

In [161]:
def find_hours_of_returning_customers():
    returning_customer_hours = {}
    #hours = list(range(0.0, 24.0))
    for customer in returning_customers.customer_id.to_list():
        order_hours = df.loc[df['customer_id'] == customer].closed_hour.to_list()
        for i in order_hours:
            try:
                if int(i) not in returning_customer_hours.keys():
                    returning_customer_hours.update({i: 1})
                else:
                    returning_customer_hours[i] +=1
            except:
                pass
    return returning_customer_hours

In [181]:
total_returning_per_hour = find_hours_of_returning_customers()

In [191]:
returning_df = pd.DataFrame(total_returning_per_hour, index = ['number']).transpose().reset_index()
returning_df_sorted = returning_df.sort_values('index')

In [207]:
final_orders = pd.merge(hourly_orders, returning_df_sorted, left_on='closed_hour', right_on='index')
final_orders.drop(['index'], axis = 1)

,closed_hour,sum,count,number
0,0.0,54508.00,4481,585
1,1.0,62859.27,5495,706
2,13.0,137.00,6,3
3,14.0,43279.00,3973,241
4,15.0,18872.50,1838,148
5,16.0,185631.40,18287,2881
6,17.0,438159.20,45940,10063
7,18.0,522385.90,54200,12533
8,19.0,233999.00,23461,4219
9,20.0,152570.80,14325,3369
